goal: save hf models from remote to local disk, so that I can populate the directory with evaluation results.

In [2]:
import platform
import sys
from huggingface_hub import snapshot_download
import os

In [3]:

save_dir = '../results/baselines/'
repo_ids = [
    # encoder-decoder
    # t5
    't5-small',
    't5-base',
    't5-large',
    't5-3b',
    't5-11b',
#     # flan-t5
    'google/flan-t5-small',
    'google/flan-t5-base',
    'google/flan-t5-large',
    'google/flan-t5-xl'
    'google/flan-t5-xxl',
#     # decoder-only
#     # 124M, 355M, 774M, 1.5B
    'gpt2',
    'gpt2-medium',
    'gpt2-large',
    'gpt2-xl',
#     # open-source
    'huggyllama/llama-7b',
    'NousResearch/Llama-2-7b-hf'
    'mosaicml/mpt-7b',
    # pythia
    'EleutherAI/pythia-70m',
    'EleutherAI/pythia-160m',
    'EleutherAI/pythia-410m',
    'EleutherAI/pythia-1b',
    'EleutherAI/pythia-1.4b',
    'EleutherAI/pythia-2.8b',
    'EleutherAI/pythia-6.9b',
    'EleutherAI/pythia-12b',
    # instruction tuned pythia
    'databricks/dolly-v2-7b',
    # 
    'allenai/open-instruct-cot-7b',
    'allenai/open-instruct-flan-v2-7b',
    'allenai/open-instruct-dolly-7b',
    'allenai/open-instruct-oasst1-7b',
    'allenai/open-instruct-human-mix-7b',
    'allenai/tulu-7b',
]
repo_ids = [
#     'mistralai/Mistral-7B-v0.1',
#     'mistralai/Mistral-7B-Instruct-v0.1',
#     'HuggingFaceH4/mistral-7b-sft-alpha',  # sft(ultrachat1.5m)
#     'HuggingFaceH4/mistral-7b-sft-beta', # sft(ultrachat200k)
    'HuggingFaceH4/zephyr-7b-alpha', # sft(ultrachat1.5m)+dpo
    'HuggingFaceH4/zephyr-7b-beta',  # sft(ultrachat200k)+dpo
]

for repo_id in repo_ids:
    snapshot_download(repo_id=repo_id, local_dir=os.path.join(save_dir, repo_id))


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

Fetching 32 files:   0%|          | 0/32 [00:00<?, ?it/s]

In [2]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="SirNeural/flan_v2", 
    repo_type="dataset",
    local_dir="../data/raw_train/flan2022",
    local_dir_use_symlinks=False)

beaker_configs	env.yml  model_licenses  requirements.txt
data		eval	 open_instruct	 results
Dockerfile	images	 quantize	 scripts
ds_configs	LICENSE  README.md	 weight-diff-requirements.txt


In [12]:
from huggingface_hub import snapshot_download

repo_id = 'HuggingFaceH4/ultrachat_200k'
local_dir = '../data/raw_train/ultrachat_200k'

snapshot_download(
    repo_id=repo_id, 
    repo_type="dataset",
    local_dir=local_dir,
    local_dir_use_symlinks=False)



Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

'/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/data/raw_train/ultrachat_200k'

In [50]:
from transformers import AutoModelForCausalLM
from transformers import MistralForCausalLM, LlamaTokenizer


p = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/mistralai/Mistral-7B-v0.1'
# model = AutoModelForCausalLM.from_pretrained(p)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")



ImportError: cannot import name 'MistralForCausalLM' from 'transformers' (/gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/wpq-llm/lib/python3.10/site-packages/transformers/__init__.py)

In [10]:
from datasets import load_dataset


ds = load_dataset(
    'HuggingFaceH4/ultrachat_200k',
    cache_dir='../data/raw_train/ultrachat',
    split='train_sft')
ds

Found cached dataset parquet (/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/scripts/../data/raw_train/ultrachat_200k/HuggingFaceH4___parquet/HuggingFaceH4--ultrachat_200k-75c6e299e27d1db5/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


Dataset({
    features: ['prompt', 'prompt_id', 'messages'],
    num_rows: 207865
})